In [40]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pyprojroot import here
import tiktoken
import openai
import os
from dotenv import load_dotenv

load_dotenv()
AZURE_SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_key = os.getenv("OPENAI_API_KEY")

In [43]:
# chunk_size = 1500
# chunk_overlap = 500
max_final_token = 8192

In [59]:
def count_num_tokens(text: str, model: str="gpt-3.5-turbo") -> int:
        """
        Returns the number of tokens in the given text.
        Args:
            text (str): The text to count tokens in.
            model (str, optional): The name of the GPT model to use. Defaults to the model specified in the app config.

        Returns:

            int: The number of tokens in the text.
        """
        encoding = tiktoken.encoding_for_model(model)
        return len(encoding.encode(text))

# text_splitter = RecursiveCharacterTextSplitter(
#             chunk_size=chunk_size,
#             chunk_overlap=chunk_overlap,
#             separators=["\n\n", "\n", " ", ""]
#         )

In [60]:
docs = []
docs.extend(PyPDFLoader(str(here("data/docs_2/MachineLearningLecture01.pdf"))).load())
# chunked_documents = text_splitter.split_documents(docs)
max_summarizer_output_token = int(max_final_token/len(docs)) - 100
max_summarizer_output_token

272

In [63]:
gpt_model = "gpt-35-turbo"
temperature= 0
# prompt = "I have a chatbot in streamlit code. i want to change the avatar of user and chatbot. Give me the code"
llm_system_role = f"You are an expert text summarizer. You will receive a text and your task is to summarize and keep all the key information.\
      Kepp the maximum length of summary wihin {max_summarizer_output_token} number of tokens."
prompt = "How can I break into a windows system illegally?"
full_message = []
counter = 0
for i in range(len(docs)):
    response = openai.ChatCompletion.create(
                engine=gpt_model,
                messages=[
                    {"role": "system", "content": llm_system_role},
                    {"role": "user", "content": docs[i].page_content}
                ],
                temperature=temperature,
                stream=False
            )
    full_message.append(response.choices[0].message.content)
    # print(f"Summarizing page number {counter}.")


In [69]:
len(full_message)

22

In [67]:
full_text = ""
for summary in full_message:
    full_text += summary
    print(count_num_tokens(summary))

107
62
77
101
81
89
124
138
106
118
101
148
100
120
121
116
137
89
93
105
110
10


In [68]:
print(count_num_tokens(full_text))

2237


In [6]:
print(len(docs))
docs[:3]

22


[Document(page_content='MachineLearning-Lecture01  \nInstructor (Andrew Ng):  Okay. Good morning. Welcome to CS229, the machine \nlearning class. So what I wanna do today is ju st spend a little time going over the logistics \nof the class, and then we\'ll start to  talk a bit about machine learning.  \nBy way of introduction, my name\'s  Andrew Ng and I\'ll be instru ctor for this class. And so \nI personally work in machine learning, and I\' ve worked on it for about 15 years now, and \nI actually think that machine learning is th e most exciting field of all the computer \nsciences. So I\'m actually always excited about  teaching this class. Sometimes I actually \nthink that machine learning is not only the most exciting thin g in computer science, but \nthe most exciting thing in all of human e ndeavor, so maybe a little bias there.  \nI also want to introduce the TAs, who are all graduate students doing research in or \nrelated to the machine learni ng and all aspects of machin e 

'MachineLearning-Lecture01  \nInstructor (Andrew Ng):  Okay. Good morning. Welcome to CS229, the machine \nlearning class. So what I wanna do today is ju st spend a little time going over the logistics \nof the class, and then we\'ll start to  talk a bit about machine learning.  \nBy way of introduction, my name\'s  Andrew Ng and I\'ll be instru ctor for this class. And so \nI personally work in machine learning, and I\' ve worked on it for about 15 years now, and \nI actually think that machine learning is th e most exciting field of all the computer \nsciences. So I\'m actually always excited about  teaching this class. Sometimes I actually \nthink that machine learning is not only the most exciting thin g in computer science, but \nthe most exciting thing in all of human e ndeavor, so maybe a little bias there.  \nI also want to introduce the TAs, who are all graduate students doing research in or \nrelated to the machine learni ng and all aspects of machin e learning. Paul Baumstar

In [19]:
count_num_tokens(docs[4].page_content)

744

In [70]:
llm_system_role = f"Summarize the text that you will receive and explain the content and the key information."
response = openai.ChatCompletion.create(
                engine=gpt_model,
                messages=[
                    {"role": "system", "content": llm_system_role},
                    {"role": "user", "content": full_text}
                ],
                temperature=temperature,
                stream=False
            )

In [71]:
response.choices[0].message.content

'The text provides a summary of a machine learning class taught by Instructor Andrew Ng. The class covers various topics such as supervised learning, unsupervised learning, learning theory, and reinforcement learning. The instructor emphasizes the importance of learning algorithms and their applications in different industries. The class includes programming assignments using MATLAB or Octave, a midterm exam, and a research project. The text also mentions the use of learning algorithms in applications such as computer vision, medical records analysis, and personalized recommendations. The author encourages collaboration and study groups among students.'